# Skupina B - Analýza výstavby dopravní infrastruktury v zemích Evropské Unie

Prvním krokem bude stažení potřebných dat. Níže jsou API odkazy na stažení dat.

In [ ]:
railway_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/rail_if_tracks$defaultview/1.0?compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'
highway_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/road_if_motorwa$defaultview/1.0?compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'
population_url = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/tps00001/1.0/*.*.*?c[freq]=A&c[indic_de]=JAN&c[geo]=EU27_2020,EA20,EA19,BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,FX,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,IS,LI,NO,CH,UK,BA,ME,MD,MK,GE,AL,RS,TR,UA,XK,AD,BY,MC,RU,SM,AM,AZ&c[TIME_PERIOD]=2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025&compress=false&format=csvdata&formatVersion=1.0&lang=en&labels=both'

Definujme funkci pro jejich stažení.

In [ ]:
import requests
import io

def download_file(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Failed to retrieve data, code: {response.status_code}')
        return None

    file = io.BytesIO(response.content)

    return file

Nyní data stáhneme a otevřeme pomocí knihovny Pandas.

In [ ]:
import pandas as pd

railway_file = download_file(railway_url)
highway_file = download_file(highway_url)
population_file = download_file(population_url)

railway_data = pd.read_csv(railway_file)
highway_data = pd.read_csv(highway_file)
population_data = pd.read_csv(population_file)

Správnost načtení ověříme zobrazením hlaviček.

In [ ]:
railway_data.head()

In [ ]:
highway_data.head()

In [ ]:
population_data.head()

**Nyní můžeme začít data analyzovat.**
Začneme tím, že z dat získáme pouze evropské státy.
Vynecháme tedy souhrnná data.

In [ ]:
population_data['geo'].unique()

Vidíme, že souhrnná data jsou tam, kde v atributu *geo* je nějaká závorka. Vybereme tedy všechna kromě těchto dat. 

In [ ]:
highway_data = highway_data[~highway_data['geo'].str.contains(r'[()]', regex=True)]
railway_data = railway_data[~railway_data['geo'].str.contains(r'[()]', regex=True)]
population_data = population_data[~population_data['geo'].str.contains(r'[()]', regex=True)]

Tím jsme se zbavili souhrnných dat.

In [ ]:
population_data['geo'].unique()

Dále rozdělíme atribut geo na kód a název státu pro sjednocení s prostorovými daty.

In [ ]:
def split_geo_column(df, source_col='geo'):
    parts = df[source_col].str.split(':', n=1, expand=True)
    df['CNTR_CODE'] = parts.iloc[:, 0]
    df['COUNTRY_NAME'] = parts.iloc[:, 1]
    return df

In [ ]:
railway_data = split_geo_column(railway_data)
highway_data = split_geo_column(highway_data)
population_data = split_geo_column(population_data)

In [ ]:
population_data.head()

Nyní veškerá data spojíme s prostorovou složkou, aby bylo možné je zobrazit v mapě. Nejprve musíme však data načíst a přichystat.

In [ ]:
import geopandas as gpd
nuts_file = "NUTS_RG_10M_2024_3035.gpkg"
nuts_data = gpd.read_file(nuts_file)

A zobrazme opět hlavičku.

In [ ]:
nuts_data.head()

Data mají nyní atribut geometry, ten využijeme pro výpočet plochy. Data mají ale pro jeden stát více úrovní. Nás ale zajímají data pouze na úrovni státu, tedy hledáme, kde je LEVL_CODE = 0.

In [ ]:
countries = nuts_data[nuts_data["LEVL_CODE"] == 0]

Nyní máme data pro celé státy, abychom zobrazili data v mapě, připojíme k němu data o infrastruktuře a počtu obyvatel. Jelikož by kolidovaly názvy atributu OBS_VALUE, přejmenujeme OBS_VALUE podle vrstev. Musíme také myslet na sjednocení dat v letech, nalezneme tedy roky, pro které máme veškerá data. Nalezneme tedy minimum maxim a maximum minim a řádky s lety mimo tento rozsah vypustíme před přejmenováním sloupců.

In [ ]:
def available_years(df):
    return set(
        df.loc[df['OBS_VALUE'].notna(), 'TIME_PERIOD'].unique()
    )

common_years = (
    available_years(railway_data)
    & available_years(highway_data)
    & available_years(population_data)
)

In [ ]:
def drop_out_of_range(df, min_year, max_year, year_col='TIME_PERIOD'):
    return df[
        df[year_col].between(min_year, max_year)
    ]

min_year = min(common_years)
max_year = max(common_years)

railway_data = drop_out_of_range(railway_data, min_year, max_year)
highway_data = drop_out_of_range(highway_data, min_year, max_year)
population_data = drop_out_of_range(population_data, min_year, max_year)

print(railway_data["TIME_PERIOD"].unique())
print(highway_data["TIME_PERIOD"].unique())
print(population_data["TIME_PERIOD"].unique())

Nyní vyřešíme kolizi názvu sloupců a přichystáme přímo data pro sloučení, abychom neměli zbytečně moc sloupců. Musíme to dělat přímo po jednotlivých letech, proto budeme rovnou iterovat přes roky.

In [ ]:
for y in common_years:
    railway_y = railway_data[railway_data['TIME_PERIOD'] == y][['CNTR_CODE', 'OBS_VALUE']].rename(columns={'OBS_VALUE': f'railway_{y}'})
    highway_y = highway_data[highway_data['TIME_PERIOD'] == y][['CNTR_CODE', 'OBS_VALUE']].rename(columns={'OBS_VALUE': f'highway_{y}'})
    population_y = population_data[population_data['TIME_PERIOD'] == y][['CNTR_CODE', 'OBS_VALUE']].rename(columns={'OBS_VALUE': f'population_{y}'})

    countries = countries.merge(railway_y, on='CNTR_CODE', how='left')
    countries = countries.merge(highway_y, on='CNTR_CODE', how='left')
    countries = countries.merge(population_y, on='CNTR_CODE', how='left')

Nyní máme přidány 3xpočet společných let nových sloupců s daty. Zobrazíme hlavičku.

In [ ]:
countries.head()

A můžeme vizualizovat v prostoru.

In [ ]:
countries.explore('railway_2015', legend=True, cmap='OrRd')

Toto bychom mohli porovnat s grafem, kolik infrastruktury mají státy průměrně na kilometr čtvereční.
Načteme tedy prostorová data o územích států.

Zobrazme, jaké mají jednotlivé státy celkovou délku infrastruktur v absolutních hodnotách.

In [ ]:
import matplotlib.pyplot as plt

def plot_absolute(df):
    plt.figure(figsize=(12, 6))
    
    for country, group in df.groupby("COUNTRY_NAME"):
        plt.plot(group["TIME_PERIOD"], group["OBS_VALUE"], label=country)
    
    plt.xlabel("Rok")
    plt.ylabel("Délka [km]")
    plt.legend(title="Státy", bbox_to_anchor=(1.05, 1), loc="right")
    plt.show()

In [ ]:
plot_absolute(highway_data)

In [ ]:
plot_absolute(railway_data)

Tím však nezjistíme nic o tom, jak se kterým státům daří. Velké státy mají více infrastruktury, vypočítejme tedy, kolik km infrastruktury připadne na 1000 obyvatel.

In [ ]:
def infrastructure_per_cap(df_infra, df_pop):
    data = pd.DataFrame({
        "geo": [],
        "TIME_PERIOD": [],
        "OBS_VALUE": []
    })

    # iterrows() iteruje přes řádky
    for _, row in df_infra.iterrows():
        geo = row["geo"]
        year = row["TIME_PERIOD"]
        infra = row["OBS_VALUE"]
        pop_row = df_pop[(df_pop["geo"] == geo) & (df_pop["TIME_PERIOD"] == year)]

        if pop_row.empty:
            continue

        population = pop_row.iloc[0]["OBS_VALUE"]

        index = infra / (population / 1000)

        new_row = pd.DataFrame({
                "geo": [geo],
                "TIME_PERIOD": [year],
                "OBS_VALUE": [index]
            })
        data = pd.concat([data, new_row], ignore_index=True)
            
    return data

In [ ]:
highway_pc = infrastructure_per_cap(highway_data, population_data)
railway_pc = infrastructure_per_cap(railway_data, population_data)

Výsledky zobrazme do grafů.

In [ ]:
def plot_index(df, index_label):
    plt.figure(figsize=(12, 6))
    
    for geo, group in df.groupby("geo"):
        plt.plot(group["TIME_PERIOD"], group["OBS_VALUE"], label=geo)
    
    plt.xlabel("Rok")
    plt.ylabel(index_label)
    plt.legend(title="Státy", bbox_to_anchor=(1.05, 1), loc="right")
    plt.show()

In [ ]:
plot_index(highway_pc, "Délka dálnic [km] na 1000 obyvatel")

In [ ]:
plot_index(railway_pc, "Délka železnic [km] na 1000 obyvatel")

Zobrazíme také histogram z posledního roku, kdy máme pro Českou republiku data, abychom získali lepší pojem o rozložení hodnot.

In [ ]:
def show_hist(df):
    plt.hist(df["OBS_VALUE"], bins=7)
    
    plt.xlabel("Hodnota")
    plt.ylabel("Počet států")
    plt.show()

Zobrazíme data pro poslední rok, kdy ještě máme data pro Českou republiku.

In [ ]:
year_hw = highway_pc[highway_pc["geo"] == "Czechia"]["TIME_PERIOD"].max()
value_hw = highway_pc[(highway_pc["geo"] == "Czechia") & (highway_pc["TIME_PERIOD"] == year_hw)]["OBS_VALUE"]

print(f"Data z roku {int(year_hw)}, Česká republika má hodnotu {value_hw.squeeze().round(5)}")

show_hist(highway_pc[highway_pc["TIME_PERIOD"] == year_hw])

In [ ]:
year_rw = railway_pc[railway_pc["geo"] == "Czechia"]["TIME_PERIOD"].max()
value_rw = railway_pc[(railway_pc["geo"] == "Czechia") & (railway_pc["TIME_PERIOD"] == year_rw)]["OBS_VALUE"]

print(f"Data z roku {int(year_rw)}, Česká republika má hodnotu {value_rw.squeeze().round(5)}")

show_hist(railway_pc[railway_pc["TIME_PERIOD"] == year_rw])

Zobrazíme ještě žebříček států v těchto ukazatelích.

In [ ]:
highway_pc_ranking = highway_pc[highway_pc["TIME_PERIOD"] == year_hw].sort_values("OBS_VALUE", ascending=False)
highway_pc_ranking

In [ ]:
railway_pc_ranking = railway_pc[railway_pc["TIME_PERIOD"] == year_rw].sort_values("OBS_VALUE", ascending=False)
railway_pc_ranking